# 데이터 병합

In [ ]:
import glob

path = "c:\\spc\\*"
flist= glob.glob(path)
flist_find = [file for file in flist if file.endswith("230222.csv")]  # 수집날짜로 해당 데이터만 불러옴

flist_find

In [ ]:
df = pd.DataFrame()

for file in flist_find:
    ftmp = pd.read_csv(file)
    df = pd.concat([df,ftmp], axis = 0)

df.head(10)

In [ ]:
df.to_csv("c:\\spc\\spclist_final_230222.csv", index = False)

# 데이터 삭제

In [ ]:
df['branch'] = df.branch.apply(lambda x: '' if x == '점' else x)

In [ ]:
# 데이터 삭제
df.drop([df[df.addr == '경남 창원시 의창구 대원동 121 1층'].index[0],df[df.branch == '창원반림일동'].index[0]], inplace = True)
df.reset_index(drop = True, inplace = True)

# Geocoding - naver api

## geocoding - 1차

In [ ]:
import requests
from tqdm.notebook import tqdm

client_id = "insert your Id"
client_secret = "insert your password"

def naver_geocoding(addr):

    # 주소
    query = addr

    endpoint = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
    url = f"{endpoint}?query={query}"

    # 헤더
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
    }

    # 요청
    res = requests.get(url, headers=headers)

    sido = 0
    sigugun = 0
    dongmyun = 0
    lat = 0
    lng = 0
    
    if res.status_code == 200:
        try:
            data = res.json()
            
            sido = data['addresses'][0]['addressElements'][0]['longName']
            sigugun = data['addresses'][0]['addressElements'][1]['longName']
            dongmyun = data['addresses'][0]['addressElements'][2]['longName']
            lat = data['addresses'][0]['y'] #위도
            lng = data['addresses'][0]['x'] #경도
        except:
            pass

    result = [sido, sigugun,dongmyun,lat, lng]

    return result

In [ ]:
df_geo = df.copy()

for idx,addr in zip(tqdm(df.index), df.addr):
    rst = naver_geocoding(addr)
    df_geo.loc[idx,'sido'] = rst[0]
    df_geo.loc[idx,'sigugun'] = rst[1]
    df_geo.loc[idx,'dongmyun'] = rst[2]
    df_geo.loc[idx,'lat'] = rst[3]
    df_geo.loc[idx,'lng'] = rst[4]


### checking - 1차
lat, lng 값이 0인 데이터들을 선별하여 네이버 지도에 지점명으로 다시 검색, 주소를 추출한다.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 

from tqdm.notebook import tqdm
import time
import requests
from bs4 import BeautifulSoup
import re
import csv

def GetDriver(url):
    driver = webdriver.Chrome("C:\\data\\chromedriver_win32\\chromedriver.exe") 
    driver.get(url)
    
    return driver


def WaitBox(driver):
    try:
        element = WebDriverWait(driver, 10).until(
           EC.presence_of_element_located((By.CLASS_NAME, "Nbox_text"))
        ) #입력창이 뜰 때까지 대기
    finally:
        pass

    
def Search(driver1,keyword):
    search_box = driver.find_element_by_xpath('//*[@id="header"]/header/div[4]/div/div/div/span[1]/input')
    search_box.click()
    search_box.send_keys(keyword)
    search_box.send_keys(Keys.ENTER)

    
def PageParser(driver):
    defhtml = driver.page_source
    defsoup = BeautifulSoup(defhtml,'html.parser') 
    
    return defsoup


def SearchNullGeo(df):
    searchdf = df.copy()

    url = "https://m.map.naver.com/search2/search.naver?query="
    
    for idx in tqdm(searchdf.index, desc = '검색중 ... '):
        keyword = f"{searchdf.loc[idx,'brand']} {searchdf.loc[idx,'branch']}"
        search_url = url+keyword

        try: 
            driver = GetDriver(search_url)

        #     WaitBox(driver)
            time.sleep(3)

            soup = PageParser(driver)

            tits = soup.find_all("div", class_ = "item_tit _title")
            addrs = soup.find_all("a", class_ = "item_address _btnAddress")

            if len(tits) > 0 : 
                re_name = ''
                re_addr = ''

                srch_name = re.sub('도넛|카페|아이스크림|베이커리','',tits[0].text)
                re_addr = re.sub('주소보기','',addrs[0].text)
                re_addr = re_addr[re.search('\w',re_addr).start():]

                searchdf.loc[idx,'search_name'] = srch_name
                searchdf.loc[idx,'addr'] = re_addr
            else:
                print(f'[No result found] index: {idx}, keyword : {keyword}')

        except:
            print(f'[FAIL] index: {idx}, keyword : {keyword}')
            continue

    driver.quit()

    print('Done')
    
    return searchdf

In [ ]:
df_cc1 = SearchNullGeo(df_geo[(df_geo.lat == 0)|(df_geo.lng == 0)])

In [ ]:
pd.set_option('display.max_rows',None) # 모든 행 열기

df_cc1   # branch랑 search_name 체크

In [ ]:
# 데이터 수정

## branch
df_cc1['branch'][(df_cc1.branch =='청주엘지로DT')&(df_cc1.brand == '파스쿠찌')] = '청주엘지로DT점'
df_cc1['branch'][(df_cc1.branch =='김천휴게소(하)')&(df_cc1.brand == '파스쿠찌')] = '김천휴게소(부산방향)점'
df_cc1['branch'][(df_cc1.branch =='김천휴게소(상)')&(df_cc1.brand == '파스쿠찌')] = '김천휴게소(서울방향)점'
df_cc1['branch'][(df_cc1.branch =='시화나래휴게소 2호점')&(df_cc1.brand == '파스쿠찌')] = '시화나래휴게소점'
df_cc1['branch'][(df_cc1.branch =='서하남휴게소점')&(df_cc1.brand == '던킨')] = '서하남휴게소'
df_cc1['branch'][(df_cc1.branch =='청주주성점')&(df_cc1.brand == '배스킨라빈스')] = '청주주성'

## addr
df_cc1['search_name'][(df_cc1.branch == '용인기흥')&(df_cc1.brand == '파리바게뜨')] = '용인기흥점'
df_cc1['addr'][(df_cc1.branch == '용인기흥점')&(df_cc1.brand == '파리바게뜨')] = '경기 용인시 기흥구 덕영대로2077번길 16'

df_cc1['addr'][(df_cc1.branch == '대유 몽베르CC')&(df_cc1.brand == '파리바게뜨')] = '경기도 포천시 영북면 산정호수로 359-12'
df_cc1['addr'][(df_cc1.branch == '염창3호')&(df_cc1.brand == '파리바게뜨')]  = '서울특별시 강서구 양천로 657'

In [ ]:
df_cc1_re = SearchNullGeo(df_cc1[df_cc1.search_name.isnull()])

### checking - 2차

In [ ]:
# 데이터 수정반영

def ChangeInfo(df_1, df_2, search_name_option):
    
    for idx in tqdm(df_2.index):
        df_1.loc[idx,'addr'] = df_2.loc[idx,'addr']
        df_1.loc[idx,'sido'] = df_2.loc[idx,'sido']
        df_1.loc[idx,'sigugun'] = df_2.loc[idx,'sigugun']
        df_1.loc[idx,'dongmyun'] = df_2.loc[idx,'dongmyun']
        df_1.loc[idx,'lat'] = df_2.loc[idx,'lat']
        df_1.loc[idx,'lng'] = df_2.loc[idx,'lng']
        
        if search_name_option == True:
            if str(df_2.loc[idx,'search_name']) == "nan" :
                df_1.loc[idx,'branch'] = df_2.loc[idx,'branch']
            else:
                br = str(df_2.loc[idx,'search_name'])
                df_1.loc[idx,'branch'] = br[br.find(' ')+1:]
        else:
            continue

In [ ]:
ChangeInfo(df_geo, df_cc1, True)
ChangeInfo(df_geo, df_cc1_re, True)

## geocoding - 2차

In [ ]:
df_cc2 = df_geo[df_geo.lat == 0] # 위경도 찾지 못한 정보만 다시 돌림
df_cc2

for idx, addr in zip(tqdm(df_cc2.index),df_cc2.addr):
    
    rst = naver_geocoding(addr)

    df_geo.loc[idx, 'sido'] = rst[0]
    df_geo.loc[idx,'sigugun'] = rst[1]
    df_geo.loc[idx,'dongmyun'] = rst[2]
    df_geo.loc[idx, 'lat'] = rst[3]
    df_geo.loc[idx,'lng'] = rst[4]
    
print("Done")

### checking - 3차

결측치 주소는 주소정보누리집 홈페이지(https://www.juso.go.kr/openIndexPage.do)에서 검색 후 업데이트

In [ ]:
df_cc3 = df_geo[df_geo.dongmyun == 0]

In [ ]:
'''
"오포읍"이 들어간 주소 전부 오류 -> 도로명주소로 입력시 오포읍 기재 필요 없음
다른 주소지의 경우 주소정보누리집이 변환해준 주소로 수정
'''
# 오포읍 수정
df_cc3.addr = df_cc3.addr.apply(lambda x: re.sub('오포읍','', str(x)))

# 주소 수정
df_cc3['addr'][df_cc3.branch == '창원시티세븐점'] = '경상남도 창원시 성산구 원이대로 320'
df_cc3['addr'][df_cc3.branch == '시흥휴게소점'] = '경기도 시흥시 수도권제1순환고속도로 105'
df_cc3['addr'][df_cc3.branch == '영통힐스테이트점'] = '경기도 수원시 영통구 망포로 142'
df_cc3['addr'][df_cc3.branch == '고덕로데오점'] = '경기도 평택시 고덕갈평5길 32'
df_cc3['addr'][df_cc3.branch == '을지로3가역점'] = '서울특별시 중구 을지로3가 282-8 을지로3구역지하쇼핑'

df_cc3

## geocoding - 3차

In [ ]:
for idx,addr in zip(tqdm(df_cc3.index), df_cc3.addr):
    
    rst = naver_geocoding(addr)
    
    df_geo.loc[idx,'addr'] = addr  # 주소 변경
    
    df_geo.loc[idx,'sido'] = rst[0]
    df_geo.loc[idx,'sigugun'] = rst[1]
    df_geo.loc[idx,'dongmyun'] = rst[2]
    df_geo.loc[idx,'lat'] = rst[3]
    df_geo.loc[idx,'lng'] = rst[4]

# Save

In [ ]:
df_geo.to_csv("c:\\spc\\spc_geo_final_230222.csv", index = False)